In [4]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import time
from math import log10
import pandas as pd
from torchvision.transforms import ToTensor, Resize
import torch.nn.functional as F
from torch.utils.data import DataLoader
import time
from dataset import load_img, convert2pilImage
from edsr import EDSR

In [5]:
model_path = "/Users/Alex2/Cours/2A/PRE/Multi-Contrast SR/best_models/t2w_t1gd_t1w->t2w 32blocsmodel_epoch_100.pth"
input_channels = ["t2w", "t1gd", "t1w"]
target_channels = ["t2w"]
n_resblocks = 32
n_patch_feats = 128
device = "cpu"
upscale_factor = 3

In [6]:
def preprocessing(data, input_channels):
    imgs = [(convert2pilImage(data[x])) for x in input_channels]
    #imgs = [Resize(80)(x) for x in imgs]
    imgs = [ToTensor()(x) for x in imgs]
    (_, h, w) = imgs[0].shape
    img_tensor = torch.stack(imgs, dim=1).view(1, len(input_channels), h, w)
    return img_tensor

def calc_psnr(pred, target):
    predd = ToTensor()(pred)
    targett = ToTensor()(target)
    print("pred: ", predd.size(), "target: ", targett.size())
    mse = nn.MSELoss()(predd, targett)
    return 10*log10(1/mse)

In [7]:
img_path = "Data/test/BRATS_034_85.npy"
data = load_img(img_path) 
hr = convert2pilImage(data["t2w"])

In [8]:
model = EDSR(upscale_factor, input_channels, target_channels, n_resblocks=n_resblocks, n_feats=n_patch_feats, res_scale=.1, bn=None).to(device)
inp = preprocessing(data, input_channels)
model.load_state_dict(torch.load(model_path, map_location="cpu"))

output = model(inp)

FileNotFoundError: [Errno 2] No such file or directory: '/best_models/t2w_t1gd_t1w->t2w 32blocsmodel_epoch_100.pth'

In [9]:
output_img = output.cpu()[0][0].detach().numpy()
norm = 255.
output_img *= norm
output_img.clip(0, 255)
output_img = Image.fromarray(output_img, mode='F').convert("L")
output_img.save("SR.jpg", quality=100)

NameError: name 'output' is not defined